---
title: Sparse Kernelized Logistic Regression
author: Evan Flaks
date: '2025-04-16'
image: "kernel.jpeg"
description: "Implementing Kernelized Logistic Regression and Performing Various Data Experiments"
format: html
---

# Implementation